In [6]:
class TableEntry:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.in_table = True

    def get_key(self):
        return self.key

    def get_value(self):
        return self.value

    def set_value(self, value):
        self.value = value

    def is_in_table(self):
        return self.in_table

    def set_to_removed(self):
        self.key = None
        self.value = None
        self.in_table = False


    def is_removed(self):
        return (self.key is None) and (self.value is None) and not self.in_table


In [7]:
class HashTableWithCount:
    """Base class for hash tables with count functionality."""
    
    def __init__(self, table_size):
        self.table_size = table_size
        self.table = [None] * table_size  # Initialize the hash table with None
        self.entry_count = 0  # Initialize entry count
        self.probe_count = 0

    def is_full(self):
        """Check if the hash table is full."""
        return self.entry_count >= self.table_size
    
    def reset_probe_count(self):
        self.probe_count = 0

    def get_probes(self):
        return self.probe_count

    def get_count(self):
        """Return the number of entries in the hash table."""
        return self.entry_count


In [8]:
class LinearProbingWithCount(HashTableWithCount):
    """Inherits HashTableWithCount using linear probing."""
    
    def primary_hash(self, key):
        """First hash function."""
        return hash(key) % self.table_size
    
    def probe(self, index):
        """Probes for an empty slot in the hash table using linear probing."""
        new_index = index
        count = 0  # Count the number of probes
        while self.table[new_index] is not None and self.table[new_index].is_in_table():
            new_index = (new_index + 1) % self.table_size
            count += 1
            if count >= self.table_size:  # Prevent infinite loop
                raise Exception("Hash table is full!")
        self.probe_count = count + 1
        return new_index

    def insert(self, key, value):
        """Inserts a key-value pair into the hash table."""
        if self.is_full():
            return
        self.reset_probe_count()
        index = self.primary_hash(key)
        if self.table[index] is not None and self.table[index].is_in_table():
            index = self.probe(index)
            
        self.table[index] = TableEntry(key, value)
        self.entry_count += 1   

    def clear(self):
        self.table = [None] * self.table_size
        self.entry_count = 0
        self.reset_probe_count()


In [9]:
class DoubleHashingWithCount(HashTableWithCount):
    """Inherits HashTableWithCount using double hashing."""
    
    def primary_hash(self, key):
        """First hash function."""
        return hash(key) % self.table_size
    
    def secondary_hash(self, key):
        """Second hash function to calculate step size."""
        return 1 + (hash(key) % (self.table_size - 1))

    def probe(self, index, key=None):
        """Probes using double hashing."""
        new_index = index
        count = 0  # Count the number of probes
        while self.table[new_index] is not None and self.table[new_index].is_in_table():
            new_index = (new_index + 1) % self.table_size
            count += 1
        
        self.probe_count = count + 1 
        return new_index
    
    def insert(self, key, value):
        """Inserts a key-value pair into the hash table."""
        if self.is_full():
            return
        self.reset_probe_count()
        index = self.primary_hash(key)
        if self.table[index] is not None and self.table[index].is_in_table():
            index = self.probe(index, key)

        self.table[index] = TableEntry(key, value)
        self.entry_count += 1   

    def clear(self):
        self.table = [None] * self.table_size
        self.entry_count = 0
        self.reset_probe_count()


In [ ]:
import random
import string
import statistics


def generate_random_names(num_names, length=8):
    names = set() 
    while len(names) < num_names: 
        name = ''.join(random.choices(string.ascii_letters, k=length)) 
        names.add(name) 
    return list(names) 

class GetStatistics: 
    def __init__(self, table_size, num_entries): 
        self.table_size = table_size 
        self.num_entries = num_entries
        self.linear_table = LinearProbingWithCount(table_size) 
        self.double_table = DoubleHashingWithCount(table_size) 

    def choose_100_names_and_add(self, linear_ht, double_ht, names): 
        chosen_names = random.sample(names, 100) 
        for name in chosen_names: 
            linear_ht.insert(name, name) 
            double_ht.insert(name, name) 
            
    def search_and_count_probes(self, linear_ht, double_ht, search_names): 
        linear_probes = [] 
        double_probes = [] 
        
        for name in search_names: 
            linear_ht.reset_probe_count() 
            double_ht.reset_probe_count()
            
            linear_ht.primary_hash(name) # Trigger search logic if necessary 
            double_ht.primary_hash(name) 
            
            linear_probes.append(linear_ht.get_probes()) 
            double_probes.append(double_ht.get_probes()) 

        return linear_probes, double_probes 
    
    def run_experiment(self, names_1000, names_10000, num_iterations=1000): 
        linear_results = [] 
        double_results = [] 

        for iteration in range(num_iterations): 
            self.linear_table.clear() 
            self.double_table.clear() 
            
            self.choose_100_names_and_add(self.linear_table, self.double_table, names_1000) 
            search_names = random.sample(names_10000, 100) 
            
            linear_probes, double_probes = self.search_and_count_probes(self.linear_table, self.double_table, search_names) 

            linear_results.append(sum(linear_probes))
            double_results.append(sum(double_probes))

            avg_linear = statistics.mean(linear_results) if linear_results else 0
            std_dev_linear = statistics.stdev(linear_results) if len(linear_results) > 1 else 0

            avg_double = statistics.mean(double_results) if double_results else 0
            std_dev_double = statistics.stdev(double_results) if len(double_results) > 1 else 0

            print(f"Average {avg_linear},  Standard devation {std_dev_linear}")
            print(f"Average {avg_double},  Standard devation {std_dev_double}")        
    
def main(): 
    table_size = 211 # Chosen size to ensure a load factor of ~0.47 (100/211) 
    num_entries = 100
    stats = GetStatistics(table_size, num_entries) 
        
    names_1000 = generate_random_names(1000) 
    names_10000 = generate_random_names(10000) 
        
    stats.run_experiment(names_1000, names_10000, num_iterations=1000) 
        
if __name__ == "__main__": 
    main()